<a href="https://colab.research.google.com/github/Flurame/MRT/blob/main/MRI_demo_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# @title 🏥 ПРОСТОЙ АНАЛИЗАТОР МРТ (Gradio)

import gradio as gr
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import img_to_array
import os
import random

print("="*60)
print("🚀 ЗАПУСК ПРОСТОГО АНАЛИЗАТОРА МРТ")
print("="*60)

# 1. ЗАГРУЗКА ИЛИ СОЗДАНИЕ МОДЕЛИ
print("\n📂 Загружаю модель...")
model_path = "/content/drive/MyDrive/BreastMRI/breast_mri_model.h5"

if os.path.exists(model_path):
    try:
        model = load_model(model_path)
        print(f"✅ Модель загружена: {model_path}")
    except:
        print("⚠️ Ошибка загрузки. Создаю новую модель...")
        model = Sequential([
            Conv2D(32, (3,3), activation='relu', input_shape=(256,256,1)),
            MaxPooling2D((2,2)),
            Conv2D(64, (3,3), activation='relu'),
            MaxPooling2D((2,2)),
            Flatten(),
            Dense(64, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
else:
    print("❌ Модель не найдена. Создаю новую...")
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(256,256,1)),
        MaxPooling2D((2,2)),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(f"📊 Модель готова. Вход: {model.input_shape}")

# 2. ФУНКЦИИ ДЛЯ РАБОТЫ С ИЗОБРАЖЕНИЯМИ
def preprocess_image(img):
    """Подготовка изображения для модели"""
    # Конвертируем в PIL если нужно
    if isinstance(img, np.ndarray):
        img = Image.fromarray(img.astype('uint8'))

    # Конвертируем в grayscale
    if img.mode != 'L':
        img = img.convert('L')

    # Изменяем размер до 256x256
    img = img.resize((256, 256), Image.Resampling.LANCZOS)

    # Подготовка для модели
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # (1, 256, 256, 1)

    return img_array, img

def get_random_image(folder_type="healthy"):
    """Получает случайное изображение из папки"""
    if folder_type == "healthy":
        folder = "/content/drive/MyDrive/BreastMRI/train/Healthy"
    else:
        folder = "/content/drive/MyDrive/BreastMRI/train/Sick"

    if os.path.exists(folder):
        files = [f for f in os.listdir(folder) if f.lower().endswith('.jpg')]
        if files:
            random_file = random.choice(files)
            img_path = os.path.join(folder, random_file)
            return Image.open(img_path)

    # Если папки нет, создаем случайное изображение
    img_array = np.random.normal(128, 30, (256, 256)).astype('uint8')
    return Image.fromarray(img_array)

# 3. ОСНОВНАЯ ФУНКЦИЯ АНАЛИЗА
def analyze_image(image):
    """Анализирует изображение и возвращает результат"""
    try:
        # Подготовка изображения
        img_array, processed_img = preprocess_image(image)

        # Предсказание
        prediction = model.predict(img_array, verbose=0)[0][0]

        # Определяем результат
        if prediction < 0.5:
            result_class = "✅ ЗДОРОВАЯ"
            confidence = (1 - prediction) * 100
            color = "green"
        else:
            result_class = "⚠️ БОЛЬНАЯ"
            confidence = prediction * 100
            color = "red"

        # Формируем результат
        result_text = f"""
        ## 🏥 РЕЗУЛЬТАТ АНАЛИЗА

        **{result_class}**

        📊 **Вероятность заболевания:** {prediction:.4f}
        🎯 **Уверенность модели:** {confidence:.1f}%

        ### 📋 Рекомендации:
        {get_recommendation(prediction)}

        ---
        *Вспомогательный инструмент. Все решения принимает врач.*
        """

        # Возвращаем результат и обработанное изображение
        return result_text, processed_img

    except Exception as e:
        return f"❌ Ошибка: {str(e)}", image

def get_recommendation(prediction):
    """Возвращает рекомендацию"""
    if prediction < 0.2:
        return "✅ Очень низкая вероятность патологии. Плановое наблюдение."
    elif prediction < 0.4:
        return "ℹ️ Низкая вероятность. Контроль через 6-12 месяцев."
    elif prediction < 0.6:
        return "⚠️ Средняя вероятность. Требуется консультация специалиста."
    elif prediction < 0.8:
        return "🚨 Высокая вероятность. Срочная консультация онколога."
    else:
        return "🆘 Очень высокая вероятность! Немедленная консультация!"

# 4. СОЗДАНИЕ ПРОСТОГО ИНТЕРФЕЙСА
print("\n🎨 Создаю интерфейс...")

with gr.Blocks(title="Анализатор МРТ", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🏥 Анализатор МРТ снимков")
    gr.Markdown("Загрузите МРТ изображение молочной железы для анализа")

    with gr.Row():
        # Левая колонка - загрузка
        with gr.Column():
            image_input = gr.Image(
                label="МРТ изображение",
                type="pil",
                height=300
            )

            with gr.Row():
                upload_btn = gr.Button("📁 Загрузить с компьютера", size="sm")
                analyze_btn = gr.Button("🔍 Анализировать", variant="primary")

            gr.Markdown("### 🎲 Быстрые тесты:")
            with gr.Row():
                healthy_btn = gr.Button("Случайное здоровое", size="sm")
                sick_btn = gr.Button("Случайное больное", size="sm")

        # Правая колонка - результаты
        with gr.Column():
            result_output = gr.Markdown(
                label="Результат анализа",
                value="**Загрузите изображение и нажмите 'Анализировать'**"
            )

            image_output = gr.Image(
                label="Обработанное изображение (256x256)",
                height=300
            )

    # ОБРАБОТЧИКИ СОБЫТИЙ
    def load_random_healthy():
        img = get_random_image("healthy")
        return img

    def load_random_sick():
        img = get_random_image("sick")
        return img

    # Привязываем кнопки
    analyze_btn.click(
        fn=analyze_image,
        inputs=[image_input],
        outputs=[result_output, image_output]
    )

    healthy_btn.click(
        fn=load_random_healthy,
        inputs=[],
        outputs=[image_input]
    )

    sick_btn.click(
        fn=load_random_sick,
        inputs=[],
        outputs=[image_input]
    )

    # Кнопка загрузки файла уже встроена в gr.Image

print("✅ Интерфейс создан!")

# 5. ЗАПУСК ПРИЛОЖЕНИЯ
print("\n🌐 Запускаю приложение...")
print("="*60)
print("⏳ Ожидайте ссылку...")

try:
    # Простой запуск
    demo.launch(share=True, debug=False)

except Exception as e:
    print(f"⚠️ Ошибка: {e}")
    print("\nПопробуйте:")
    print("1. Перезапустить runtime")
    print("2. Запустить без share:")
    print("   demo.launch()")
    print("\nИли попробуйте этот упрощенный запуск:")
    demo.launch()

🚀 ЗАПУСК ПРОСТОГО АНАЛИЗАТОРА МРТ

📂 Загружаю модель...


/tmp/ipython-input-53129764.py:148: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="Анализатор МРТ", theme=gr.themes.Soft()) as demo:


✅ Модель загружена: /content/drive/MyDrive/BreastMRI/breast_mri_model.h5
📊 Модель готова. Вход: (None, 256, 256, 1)

🎨 Создаю интерфейс...
✅ Интерфейс создан!

🌐 Запускаю приложение...
⏳ Ожидайте ссылку...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a7ef078e7ae6679c78.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
